# Converting the csv files to a SQL database

In [29]:
import pandas as pd
import sqlite3
import os
import re

In [30]:
%load_ext autoreload
%autoreload 2

# my code
from SQLutils import addCSVToSQLdb, createNewSQLdbFromCSVs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create new database files with the desired paragraph(s)

In [31]:
allPara = pd.read_csv('../csv/paragraphs.csv')
allParaName = allPara['paragraphname'].to_list()
allParaName

['Polymer Composite Example',
 'Aluminum-CHiMaDExample',
 'Thermoelectrics_test',
 'Emery-Exam1-M54',
 'blank',
 'Emery-Exam1-M54_1',
 'New Paragraph',
 'Polymer Composite Example 1',
 'CJE DSA Random BCPs',
 'whitney DSA',
 'Co_alloy_WhitneyT',
 'Metal AM - SLM',
 'Thermoelectrics_JM',
 'Superalloy -test',
 'Cobalt-based Superalloys_Brandon',
 'CoAlloy_WhitneyT',
 'Cobalt-Based Superalloys_Brandonfinished',
 'CJE BCPs for DSA',
 'Co-based Superalloys Brandon3',
 'Co-based superalloys Brandon4',
 'PECs',
 'PECs_class',
 'test',
 'dsa_finalmaybe',
 'testing1234',
 'testing12345']

In [32]:
#all that we currently have
createNewSQLdbFromCSVs('Dec2021_all', allParaName)

file does not exist ../csv/testing1234.csv
file does not exist ../csv/testing12345.csv


In [33]:
#default, available on load
default = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54']
createNewSQLdbFromCSVs('default', default)

Can't remove file  default.db


In [34]:
createNewSQLdbFromCSVs('testing', default)

In [35]:
useCases = ['Polymer Composite Example','Aluminum-CHiMaDExample','Thermoelectrics_test','Emery-Exam1-M54',
    'CJE BCPs for DSA',
    'Co_alloy_WhitneyT',
    'Cobalt-Based Superalloys_Brandonfinished',
    'Metal AM - SLM',
    'PECs',
    'PECs_class',
    'Superalloy -test',
    'Thermoelectrics_JM',
    'whitney DSA',
    'CJE DSA Random BCPs'
]
createNewSQLdbFromCSVs('CHiMaD_UseCases', useCases)

# Create the metrics database

In [2]:
dbName = 'CHiMaD_metrics.db'
tableName = 'loginMetrics'
columns = ['timestamp', 'name', 'organization', 'email', 'purpose', 'groupname']

In [3]:
# connect to the SQL database (this will create a file if it does not already exist)
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

# include all columns, and assume that they are all text
# allow only alphanumeric values in column names
cols = ''
for col in columns:
    cc = re.sub(r'\W+', '', col) 
    cols += cc + ' text, ' 
cols = cols[:-2]
    
# create the table
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

cursor.close()

# Remove a table from a file

In [27]:
dbName = 'default.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

dropTableStatement = "DROP TABLE dbs"
cursor.execute(dropTableStatement)
cursor.close()

# Remove a row from a database file

In [28]:
#read in table 
dbName = 'default.db'
tablename = 'polymercompositeexample'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

# perform selection
#df = df.loc[~((df['username'] == 'AaronGeller') & (df.index > 41))]
# rewrite table
#df.to_sql(tablename, conn, if_exists='replace', index = False)
#display(df)

cursor.close()

['paragraphs', 'aluminumchimadexample', 'thermoelectricstest', 'emeryexam1m54', 'polymercompositeexample']


,Timestamp,IP,username,version,task,lightweightmechanical,toughness,strengthstiffness,lowdensity,extremeenvironments,...,thechoiceandpreparationofreinforcement,morphology,microstructureofthereinforcementphase,layupandcuring,dispersion,aggregation,orientationofthereinforcement,postlayupannealingandaging,matrixreinforcementinterface,failuremode
0,11/1/2021 9:29:25,00.000.00.00,JonEmery,1,para,Property,Property,Property,Property,Structure,...,Property,Property,Property,Structure,Property,Structure,Property,Performance,Structure,Processing
1,11/1/2021 13:41:39,00.000.00.00,JonEmery,2,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
2,11/1/2021 9:33:38,00.000.00.00,JonEmery,1,SDC,None,resistancetofracture,lightweightmechanical,lightweightmechanical,None,...,microstructureofthereinforcementphase heteroge...,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,aggregation microstructureofthereinforcementphase,slowdegradationrateswhenexposedtoo2h2oandsalts,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,matrixreinforcementinterface aggregation,failuremode,resistancetofracture
3,11/1/2021 13:15:01,00.000.00.00,WhitneyT,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
4,11/1/2021 13:15:09,00.000.00.00,chris_h,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Performance
5,11/1/2021 13:15:12,00.000.00.00,AaronGeller,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
6,11/1/2021 13:16:08,00.000.00.00,Alex,1,para,Performance,Property,Property,Property,Performance,...,Processing,Property,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Structure
7,11/1/2021 13:16:09,00.000.00.00,JamesMale,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
8,11/1/2021 13:17:18,00.000.00.00,BrandonO,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Structure,Property
9,11/1/2021 13:17:35,00.000.00.00,Kuma,1,para,Performance,Property,Property,Property,Performance,...,Processing,Structure,Structure,Processing,Structure,Structure,Structure,Processing,Processing,Property


## Test selecting everything from the database and puting back into DataFrame

In [9]:
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]

for t in tables:
    print(t)
    cursor.execute('SELECT * FROM ' + t)
    columns = [description[0] for description in cursor.description]
    df2 = pd.DataFrame(cursor.fetchall(), columns = columns)    
    display(df2)


testing1234


,Timestamp,IP,username,version,task,lightweightmechanical,toughness,strengthstiffness,lowdensity,extremeenvironments,slowdegradationrateswhenexposedtoo2h2oandsalts,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,foo,bar
0,12/01/2021 18:09:33,00.000.00.00,AaronGeller,1,para,Processing,Property,Processing,Processing,Structure,Structure,Property,Processing,Property,Performance
1,12/01/2021 18:10:12,00.000.00.00,AaronGeller,2,para,Processing,Property,Structure,Structure,Property,Property,Property,Structure,Performance,Performance
2,12/01/2021 18:10:30,00.000.00.00,AaronGeller,1,SDC,glasstransitiontemperaturetg,bar,,,,,extremeenvironments,,,
3,12/01/2021 18:10:56,00.000.00.00,AaronGeller,2,SDC,glasstransitiontemperaturetg lowdensity,bar,slowdegradationrateswhenexposedtoo2h2oandsalts,extremeenvironments,foo,bar,extremeenvironments slowdegradationrateswhenex...,toughness,,
4,12/01/2021 18:12:08,00.000.00.00,AaronGeller0,1,para,Processing,Property,Structure,Structure,Property,Property,Property,Structure,Performance,Performance
5,12/01/2021 18:15:55,00.000.00.00,AaronGeller0,2,para,Processing,Property,Structure,Structure,Property,Property,Property,Structure,Performance,Performance
6,12/01/2021 18:12:14,00.000.00.00,AaronGeller0,1,SDC,glasstransitiontemperaturetg lowdensity,bar,slowdegradationrateswhenexposedtoo2h2oandsalts,extremeenvironments,foo,bar,extremeenvironments slowdegradationrateswhenex...,toughness,,
7,12/01/2021 18:15:59,00.000.00.00,AaronGeller0,2,SDC,glasstransitiontemperaturetg lowdensity therma...,bar,slowdegradationrateswhenexposedtoo2h2oandsalts,extremeenvironments,foo,bar foo,extremeenvironments slowdegradationrateswhenex...,toughness,,
8,12/01/2021 18:20:28,00.000.00.00,AaronGeller2,1,para,Processing,Property,Property,Structure,Performance,Structure,Structure,Processing,Structure,Performance
9,12/01/2021 18:20:37,00.000.00.00,AaronGeller2,2,para,Processing,Property,Structure,Structure,Performance,Structure,Structure,Processing,Structure,Performance


testing12345


,Timestamp,IP,username,version,task,lightweightmechanical,toughness,strengthstiffness,lowdensity,extremeenvironments,slowdegradationrateswhenexposedtoo2h2oandsalts,thermalexpansioncoefficientmismatch,glasstransitiontemperaturetg,foo,bar,fee


cjebcpsfordsa


...

,Timestamp,IP,username,version,task,ingotmetallurgy,ingotrefining,deoxidation,ingotsolidification,feandotherimpuritiesformcoarseinclusionorconstituentphasessuchasal3fe,...,hotworking,solutiontreatment,quenched,aged,precipitatecugpzones,finalmatrix,elasticmodulus,yieldstrength,fracturetoughness,grainsize


emeryexam1m541


,Timestamp,IP,username,version,task,figureofmeritzt,latticethermalconductivity,electronicthermalconductivity,carriermobility,carrierconcentration,...,densityofstates,dopingthematrix,latticethermalconductivityresistance,microstructuregrainboundariessecondaryphases,castthealloyasaningot,consolidationhomogenizationstepiesparkplasmasinteringhotpressing,annealingtreatment,grainstructureincludinggbchemistry,nanostructureofthesecondaryphases,phononscattering


paragraphs


,paragraphname,paragraph,answersJSON
0,Polymer Composite Example,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample"",""t..."
1,Aluminum-CHiMaDExample,An aluminum alloy is prepared by [[ingot metal...,"[{""paragraphname"":""aluminumchimadexample"",""tas..."
2,Thermoelectrics_test,The efficiency of thermoelectric devices is re...,"[{""paragraphname"":""thermoelectricstest"",""task""..."
3,Emery-Exam1-M54,"In 2007, the US Navy contracted QuestTek to de...","[{""paragraphname"":""emeryexam1m54"",""task"":""para..."
4,blank,None,None
5,Emery-Exam1-M54_1,The efficiency of thermoelectric devices is re...,None
6,New Paragraph,This is a [[great]] paragraph [[with]] lots of...,"[{""paragraphname"":""newparagraph"",""task"":""para""..."
7,Polymer Composite Example 1,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample1"",""..."
8,CJE DSA Random BCPs,Polymer matrix composites are traditionally va...,None
9,whitney DSA,There are several material design requirements...,None


In [16]:
cursor.close()

# Manual testing

## Read in a csv file

In [12]:
#csvName = 'thermoelectricsjm.csv' # this has a column with ".1" in the name, and sqlite does not allow it
csvName = 'paragraphs.csv' # this has a column with ".1" in the name, and sqlite does not allow it
df = pd.read_csv('../csv/' + csvName)
df

,paragraphname,paragraph,answersJSON
0,Polymer Composite Example,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample"",""t..."
1,Aluminum-CHiMaDExample,An aluminum alloy is prepared by [[ingot metal...,"[{""paragraphname"":""aluminumchimadexample"",""tas..."
2,Thermoelectrics_test,The efficiency of thermoelectric devices is re...,"[{""paragraphname"":""thermoelectricstest"",""task""..."
3,Emery-Exam1-M54,"In 2007, the US Navy contracted QuestTek to de...","[{""paragraphname"":""emeryexam1m54"",""task"":""para..."
4,blank,NaN,NaN
5,Emery-Exam1-M54_1,The efficiency of thermoelectric devices is re...,NaN
6,New Paragraph,This is a [[great]] paragraph [[with]] lots of...,"[{""paragraphname"":""newparagraph"",""task"":""para""..."
7,Polymer Composite Example 1,Polymer matrix composites are traditionally va...,"[{""paragraphname"":""polymercompositeexample1"",""..."
8,CJE DSA Random BCPs,Polymer matrix composites are traditionally va...,NaN
9,whitney DSA,There are several material design requirements...,NaN


## Create a SQL database

In [10]:
dbName = 'CHiMaD_SDC.db'
conn = sqlite3.connect(dbName)
cursor = conn.cursor()

In [13]:
# the database name will be the file name w/o csv
# take only alphanumeric values
tableName = re.sub(r'\W+', '',csvName.replace('.csv','')) 
tableName

'paragraphs'

In [ ]:
# include all columns, and assume that they are all text
cols = ''
for col in df.columns:
    cc = re.sub(r'\W+', '', col) # only alphanumeric values in column names
    cols += cc + ' text, '
cols = cols[:-2]
cols

In [ ]:
cursor.execute('CREATE TABLE IF NOT EXISTS ' + tableName + ' (' + cols + ')')
conn.commit()

In [ ]:
# add the dataframe into the database
df.to_sql(tableName, conn, if_exists='replace', index = False)

In [15]:
#grab one table and write to file
t = 'paragraphs'
cursor.execute('SELECT * FROM ' + t)
columns = [description[0] for description in cursor.description]
df2 = pd.DataFrame(cursor.fetchall(), columns = columns)   
df2.to_csv(t+'.csv', index=False)

In [57]:
#read in table to check
# dbName = 'CHiMaD_metrics.db'
# tablename = 'loginMetrics'
#dbName = 'uchicago.db'
#dbName = 'dec2021_all.db'
#tablename = 'paragraphs'
#tablename = 'polymercompositeexample'
dbName = 'available_dbs.db'
tablename = 'dbs'
# dbName = 'default.db'
# tablename = 'polymercompositeexample'

conn = sqlite3.connect(dbName)
cursor = conn.cursor()
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [x[0] for x in cursor.fetchall()]
print(tables)

cursor.execute('SELECT * FROM ' + tablename)
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(cursor.fetchall(), columns = columns)  
display(df)

cursor.close()

['dbs']


,groupname
0,Dec2021_all
1,default
2,testing
3,CHiMaD_UseCases
4,foo


In [56]:
display(df)
df.replace({'groupname': {'testing': 'foo'}}, inplace = True)
display(df)

,groupname
0,Dec2021_all
1,default
2,testing
3,CHiMaD_UseCases
4,testing1


,groupname
0,Dec2021_all
1,default
2,foo
3,CHiMaD_UseCases
4,testing1


In [8]:
cursor.close()

In [ ]:
# command to copy files from Downloads and change names
#find ~/Downloads/ | grep testParagraph | awk '{print ("cp "$0" "$3)}' | sed s/" - "/"\\\ -\\\ "/ > foo